In [1]:
# list ids: 
ids=['00000']

for i in range(1,27639):
    num=''
    if i <10:
        num+='0000'
        num+=str(i)
    elif i<100:
        num+='000'
        num+=str(i)        
    elif i<1000:
        num+='00'
        num+=str(i)       
    elif i<10000:
        num+='0'
        num+=str(i)  
    else:
        num+=str(i)
    ids.append(num)
#print(ids)

In [2]:
# pipeline to transform an image
from PIL import Image
import torchvision.transforms as transforms
from torch import Tensor
import torch.utils
import numpy as np

scaler=transforms.Resize((224,224))
normalize=transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor=transforms.ToTensor()

def transform_image(image_id):
    image=Image.open("Yummly28K\images27638\img" + image_id +'.jpg')
    t_img=normalize(to_tensor(scaler(image))).unsqueeze(0)
    

    return t_img.cpu().detach().numpy()

transform_image('00001').shape
#type(transform_image('00001'))

(1, 3, 224, 224)

In [3]:
file = open("one_hot_meat.csv", 'r')
l = file.readlines()
file.close()
one_hot = []
dummy = [1] * 12
one_hot.append(dummy)
for item in l:
    temp = item.strip('\n')
    temp = temp.split(',')
    n = len(temp)
    for i in range(n):
        temp[i] = int(temp[i])
    one_hot.append(temp)

#print(one_hot[0], one_hot[1])
# index 1 is 1st y hot

In [4]:
X_train_normalized_list=[]
y_train_list=[]

X_val_normalized_list=[]
y_val_list=[]

for i in range(1,2501):
    image=transform_image(ids[i])
    X_train_normalized_list.append(image)
    y_train_list.append(one_hot[i])
    if i%100==0:
        print(i)

for i in range(2501, 3001):
    image=transform_image(ids[i])
    X_val_normalized_list.append(image)
    y_val_list.append(one_hot[i])
    if i%100==0:
        print(i)   



100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000


In [5]:
n_iterations=40
batch_size=250

X_train_normalized=np.array(X_train_normalized_list) 
print("completed")
y_train=np.array(y_train_list)
print("completed")
#rint(X_train_normalized.shape())
#print(y_train_normalized.shape())

X_val_normalized=np.array(X_val_normalized_list)    
y_val=np.array(y_val_list)

n_data=X_train_normalized.shape[0]
n_batch=int(np.ceil(n_data/batch_size))

X_train_tensor=Tensor(X_train_normalized)
X_val_tensor=Tensor(X_val_normalized)
X_train_normalized =[]
X_val_normalized = []
print("clear memory")

y_train_tensor=Tensor(y_train).float()
y_val_tensor=Tensor(y_val).float()


  

completed
completed
clear memory


In [6]:
print(X_train_tensor.shape)
print(X_val_tensor.shape)
print(y_train_tensor.shape)
print(y_val_tensor.shape)


torch.Size([2500, 1, 3, 224, 224])
torch.Size([500, 1, 3, 224, 224])
torch.Size([2500, 12])
torch.Size([500, 12])


In [7]:
epochs=5
use_gpu=torch.cuda.is_available()
import torchvision.models as models
import copy
import torch.optim as optim
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

In [8]:
def train_model(model,criterion,optimizer,epochs=5):
    best_model_w=copy.deepcopy(model.state_dict())
    best_accuracy=0.0
    worst_x_index = 0
    worst_prediction = []
    worst_acccuracy = 1.0
    y_actual = []
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)
        train_loss = 0.0
        train_accuracy = 0.0
        val_loss = 0.0
        val_accuracy = 0.0
        for phase in ['train','valid']:
            if phase=='train':
                model.train(True)
            else:
                model.train(False)


            if phase=='train':
                for i in range(0,2500):
                    if i % 100 == 0:
                        print(i)
                    inputs=X_train_tensor[i]
                    labels=y_train_tensor[i]

                    if use_gpu==True:
                        inputs=Variable(inputs.cuda())
                        labels=Variable(labels.cuda())
                    else:
                        inputs, labels = Variable(inputs),Variable(labels)
                    optimizer.zero_grad()
                    outputs=model(inputs)
                    #print("this is outputs:", outputs[0])
                    #print("this is labels:", labels)
                    loss=criterion(outputs[0], labels)

                    loss.backward()
                    optimizer.step()

                    train_loss+=(loss.data.mean())
                    predict = make_prediction(outputs[0])
                    acc = check_correctness(predict,labels)
                    train_accuracy += acc
                    #if acc < worst_acccuracy:
                    #    worst_x_index = i
                    #    worst_prediction = predict
                    #    y_actual = labels
                    if acc == 1 and labels[-1] != 1:
                        worst_x_index = i
                        worst_predicion = 0
                        y_actual = labels
                
                train_accuracy = train_accuracy/2500
                if train_accuracy > best_accuracy:
                    best_accuracy = train_accuracy
                    best_model_w = copy.deepcopy(model.state_dict())
                    

            else:
                for j in range(0, 500):
                    #print(j)
                    inputs=X_val_tensor[j]
                    labels=y_val_tensor[j]
                    
                    
                    if use_gpu==True:
                            inputs=Variable(inputs.cuda())
                            labels=Variable(labels.cuda())
                    else:
                        inputs, labels = Variable(inputs),Variable(labels)
                    outputs=model(inputs)
                    loss=criterion(outputs[0],labels)
                    val_loss+=(loss.data.mean())
                    predict = make_prediction(outputs[0])
                    val_accuracy += check_correctness(predict, labels)
                    
                val_accuracy = val_accuracy/500
                
        print("For Epoch number:", epoch, "Phase: training, total loss: ", train_loss, "Average_Accuracy:", train_accuracy)
        print("For Epoch number:", epoch, "Phase: validation, total loss: ", val_loss, "Average_Accuracy:", val_accuracy)

    
    print("the worst one is located at: ", worst_x_index)
    print("the guess was:", worst_prediction)
    print("The actual was:" , y_actual)
    model.load_state_dict(best_model_w)
    return model
            



In [9]:
import random

def make_prediction(tensor):
    n = len(tensor)
    status=False
    for i in range(n):
        if tensor[i] > .5:
            tensor[i] = 1
            status=True
        elif tensor[i] < .5:
            tensor[i] = 0
        else:
            tensor[i] = random.randint(0,1)
            
    if (status == False):
        tensor[-1] = 1
            
    return tensor
    
def check_correctness(y_pred, y_actual):
    n = len(y_pred)
    nb_correct = 0
    nb_incorrect = 0
    nb_target = 0
    for i in range(n):
        if y_actual[i] == 1:
            if y_pred[i] == 1:
                nb_correct += 1
            #false negative
            else:
                nb_incorrect += 1
            nb_target += 1
        #y_pred == 0
        else:
            #false positive
            if y_pred[i] == 1:
                nb_incorrect += 1
        
    accuracy = nb_correct/(nb_correct + nb_incorrect)
    return accuracy

In [10]:
import torchvision.models as models
import copy
import torch.optim as optim
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


resnet50 = models.resnet50()

model = models.resnet50(pretrained=True)

## will need to replace last layer of model into fully conneted softmax
#  last layer of resnet (fc): Linear(in_features=2048, out_features=1000, bias=True)




# 12 for length 12 of meat 
#model.fc=torch.nn.Sequential(torch.nn.Linear(in_features=2048, out_features=12, bias=True),torch.nn.Softmax())

state_dic=torch.utils.model_zoo.load_url("https://download.pytorch.org/models/resnet50-19c8e357.pth", progress=True)
model.load_state_dict(state_dic)
model.eval()

#resnet50_weights=copy.deepcopy(state_dic)


#print(state_dic)
#print(model.state_dict())

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [15]:
use_gpu=torch.cuda.is_available()
print(use_gpu)

model = models.resnet50(pretrained=True)

#I recommend training with these layers unfrozen for a couple of epochs after the initial frozen training
count = 0
retrain = 125
for param in model.parameters():
    if count < retrain:
        param.requires_grad = False
    else:
        param.requires_grad = True
    count +=1
numb_ftrs = model.fc.in_features
model.fc=torch.nn.Linear(in_features=numb_ftrs, out_features=12, bias=True)

#num_ftrs = model.fc.in_features
#model.fc = torch.nn.Linear(num_ftrs, len(classes))
if use_gpu:
    model = model.cuda()

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=.001)

#optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#loaders = {'train':train_loader, 'valid':valid_loader, 'test': test_loader}

True


In [19]:
count = 0
t = 0
f = 0
for param in model.parameters():
    count += 1
    if param.requires_grad == True:
        t += 1
    else:
        f += 1
print(count)
print(t)
print(f)

161
36
125


In [20]:
model = train_model(model, criterion, optimizer)

Epoch 0/4
----------
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
For Epoch number: 0 Phase: training, total loss:  tensor(2131.7910, device='cuda:0') Average_Accuracy: 0.4706466666666666
For Epoch number: 0 Phase: validation, total loss:  tensor(237.4136, device='cuda:0') Average_Accuracy: 0.602
Epoch 1/4
----------
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
For Epoch number: 1 Phase: training, total loss:  tensor(2058.8735, device='cuda:0') Average_Accuracy: 0.4709
For Epoch number: 1 Phase: validation, total loss:  tensor(236.9566, device='cuda:0') Average_Accuracy: 0.592
Epoch 2/4
----------
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
For Epoch number: 2 Phase: training, total loss:  tensor(2041.2578, device='cuda:0') Average_Accuracy: 0.4761333333333334
For Epoch number: 2 Phas

In [ ]:
model.eval()
# make test data
# take 
X_test_normalized_list=[]
y_test_list=[]

for i in range(3001, 3501):
    image=transform_image(ids[i])
    X_test_normalized_list.append(image)
    y_test_list.append(one_hot[i])  
        
X_test_normalized=np.array(X_test_normalized_list)

y_test=np.array(y_test_list)
#y_test_tensor=Tensor(y_test).float()

data=Tensor(X_test_normalized)
actual=Tensor(y_test).float()

sum_acc=0
for i in range(500):
    inputs=data[i].cuda()
    output=model(inputs)
    prediction=make_prediction(output[0])
    actuals=actual[i].cuda()
    acc=check_correctness(prediction,actuals)
    sum_acc+=acc


overall_acc=sum_acc/500
print(overall_acc)


  


In [ ]:
def test_1(img_id):
    img = transform_image(img_id)
    x = Tensor(img)
    y_val = one_hot[int(img_id)]
    y_val = Tensor(y_val)
    if use_gpu==True:
        inputs=Variable(x.cuda())
        labels=Variable(y_val.cuda())
    else:
        inputs, labels = Variable(x.cuda()),Variable(y_val.cuda())
    result = model(inputs)
    prediction = make_prediction(result[0])
        
    print(y_val)
    print(prediction)
test_1("02496")